In [ ]:
# Install pyspark
! pip install --ignore-installed pyspark

# Install Spark NLP
! pip install --ignore-installed spark-nlp

This notebook test serialization and LightPipeline for EntityRuler

In [ ]:
import sparknlp
from sparknlp.base import *
from sparknlp.annotator import *
from pyspark.sql import SparkSession

In [ ]:
print("Spark NLP version", sparknlp.version())

Apache Spark version: 3.0.1


In [ ]:
data = spark.createDataFrame([[""]]).toDF("text")

In [ ]:
entity_ruler = EntityRulerApproach() \
    .setInputCols(["document", "token"]) \
    .setOutputCol("entity") \
    .setPatternsResource("sample_data/patterns.json")

entity_ruler_model = entity_ruler.fit(data)
entity_ruler_model.write().overwrite().save("tmp_entity_ruler_model")

In [ ]:
entity_ruler_loaded = EntityRulerModel().load("tmp_entity_ruler_model")

In [ ]:
document_assembler = DocumentAssembler().setInputCol("text").setOutputCol("document")
tokenizer = Tokenizer() \
  .setInputCols("document") \
  .setOutputCol("token") \
  .setExceptions(["John Snow", "Eddard Stark"])

pipeline = Pipeline(stages=[document_assembler, tokenizer, entity_ruler])
pipeline_model = pipeline.fit(data)
light_pipeline = LightPipeline(pipeline_model)

In [ ]:
result = light_pipeline.annotate("Lord Eddard Stark was the head of House Stark. John Snow lives in Winterfell.")
print(result)

{'document': ['Lord Eddard Stark was the head of House Stark. John Snow lives in Winterfell.'], 'token': ['Lord', 'Eddard Stark', 'was', 'the', 'head', 'of', 'House', 'Stark', '.', 'John Snow', 'lives', 'in', 'Winterfell', '.'], 'entity': ['Eddard Stark', 'John Snow', 'Winterfell']}
